In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Data generation
We generated the data using the correct model assumed.

In [ ]:
# Here we also provide the function we used to generate the dataset.
def generate_data():
    v_noise_y = .4
    v_noise_x = .01
    n = 100
    w_true = [1,-2,0,1,4]
    rs = np.random.RandomState(110)
    
    x = np.linspace(-1,1,n) + rs.randn(n)*np.sqrt(v_noise_x)
    x[x<-1] = -2 - x[x<-1]
    x[x>1] = 2 - x[x>1]
    
    ps = np.polynomial.chebyshev.Chebyshev(w_true)
    y_true = ps(x)
    y = y_true + rs.randn(n)*np.sqrt(v_noise_y)
    p = np.argsort(x)
    return x[p],y[p],y_true[p]
x,y,y_true = generate_data()
np.savetxt('data/y_target.csv',np.c_[x,y],delimiter=',')

In [ ]:
plt.plot(x,np.c_[y,y_true])

### Loading data
We first load the problem data.

In [ ]:
def load_data():
    xy = np.loadtxt('data/y_target.csv', delimiter=",")
    return xy[:,0],xy[:,1]

x,y = load_data()
plt.plot(x,y)

In [ ]:
def generate_features(x, deg):
    '''This function generates a design matrix of features for each input point in x
    
    @param x: the input points
    @param deg: the maximum degree of the polynomial basis.
    @return : the design matrix X of dimensions NxD+1, so that the i-th row is the feature vector of the i-th input.
              Each value X_id = f_d(x_i) for the Chebyshev polynomial (of 1st kind) with degree d.
    '''
    X = np.polynomial.chebyshev.chebvander(x, deg)
    return X

D = 4
X = generate_features(x, D)
X.shape

In [ ]:

def fit(X,y):
    '''Learns the coefficients of each of the features in the provided matrix that best predicts y.
    @param X: the design matrix of features, one feature per row
    @param y: the vector of the dependent variable (labels)
    @return: vector of coefficients
    '''
    #Fit using least squares and return the array of coefficients w of dimension deg+1
    w = np.linalg.inv(X.T@X)@X.T@y
    return w

def fit_cheb(x,y,deg):
    '''Learns the coefficients of each of the features in the provided matrix that best predicts y.
    @param x: the input points
    @param deg: maximum depgree of chebyshev polynomials
    @param y: the vector of the dependent variable (labels)
    @return: vector of coefficients
    '''
    X = generate_features(x, deg)
    return fit(X, y)
    
w = fit_cheb(x, y, D)
w, X.shape

In [ ]:
def predict(X,w):
    '''Predicts the labels of the linear model using the given coefficients.
    @param X: the design matrix of features, one feature per row
    @param w: the vector of coefficients
    @return: vector of the predicted dependent variable
    '''
    return X @ w

def predict_cheb(x,deg,w):
    '''Predicts the labels of our specific model using the given coefficients.
    @param x: the input points
    @param deg: maximum depgree of chebyshev polynomials
    @param w: the vector of coefficients
    @return: vector of the predicted dependent variable
    '''
    X = generate_features(x, deg)
    return predict(X, w)
    
yhat = predict_cheb(x,D,w)
plt.plot(x, np.c_[y, yhat])


In [ ]:
from utilities import split_data

# Now generate a split of the full data into a taining/.testing dataset.
# The result is an object with named attributes x_trn, x_tst, t_trn, and y_tst.
data = split_data(x, y)

In [ ]:
# Fit on the train data and evalute the RSS on the test data

def mse(y,y_pred):
    '''Compute the mean squared error of a prediction and its true label.
    @param y: vector of true labels
    @param y_hat: vector of predictions
    @return: the MSE
    '''
    return np.mean((y_pred-y)**2)
    
def evaluate_model_on_dataset(data, deg):
    '''Evaluate our model on the given training/testing set.
    @param data: The object holding the current split.
    @param deg: maximum depgree of chebyshev polynomials
    @return: the MSE of the predictions returned by the model learned on the training data
             as computed against on the testing labels.
    '''
    X_train = generate_features(data.x_trn, deg)
    X_test = generate_features(data.x_tst, deg)
    w = fit(X_train, data.y_trn)
    y_pred = predict(X_test, w)
    return mse(data.y_tst, y_pred)
    

### Evaluation
We now evaluate our models for different degrees.

In [ ]:
degs = np.arange(15)
MSEs = np.r_[[evaluate_model_on_dataset(data, deg) for deg in degs]]

In [ ]:
plt.bar(degs, MSEs)
plt.xlabel('Maximal Degree')
plt.ylabel('Mean Squared Error')

In [ ]:
from utilities import split_data_around_point
data_ap = split_data_around_point(x, y, x_0=0.9)

### Evaluation (splits)
We compare the effect of the two different splits on the generalisation error.

In [ ]:
MSEs_ap = np.r_[[evaluate_model_on_dataset(data_ap, deg) for deg in degs]]
plt.bar(degs+.2, MSEs, width=.4)
plt.bar(degs-.2, MSEs_ap, width=.4)
plt.xlabel('Maximal Degree')
plt.ylabel('Mean Squared Error')